# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
nltk.download(['punkt','wordnet','stopwords'])

random_state = 99

[nltk_data] Downloading package punkt to /home/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
X = df['message'] 
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()    
    text = re.sub(r"[^a-zA-Z]", " ", text.lower()) 
    tokens = word_tokenize(text)
    
    return [lemmatizer.lemmatize(t).strip() for t in tokens if t not in stopwords.words("english")]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(RandomForestClassifier(random_state=random_state)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = random_state)

In [7]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred_train = pipeline.predict(X_train)
print(classification_report(Y_train.values, y_pred_train, target_names=Y.columns.values))

In [9]:
y_pred_test = pipeline.predict(X_test)
print(classification_report(Y_test.values, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               request       0.82      0.50      0.62       902
                 offer       0.00      0.00      0.00        22
           aid_related       0.76      0.68      0.72      2187
          medical_help       0.52      0.06      0.10       444
      medical_products       0.76      0.15      0.25       256
     search_and_rescue       0.71      0.04      0.07       129
              security       0.00      0.00      0.00       100
              military       0.77      0.05      0.10       197
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.38      0.53       306
                  food       0.86      0.59      0.70       578
               shelter       0.83      0.38      0.52       453
              clothing       0.89      0.10      0.18        81
                 money       0.71      0.04      0.08       117
        missing_people       0.00      

/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
sorted(pipeline.get_params().keys())

['memory',
 'moc',
 'moc__estimator',
 'moc__estimator__bootstrap',
 'moc__estimator__ccp_alpha',
 'moc__estimator__class_weight',
 'moc__estimator__criterion',
 'moc__estimator__max_depth',
 'moc__estimator__max_features',
 'moc__estimator__max_leaf_nodes',
 'moc__estimator__max_samples',
 'moc__estimator__min_impurity_decrease',
 'moc__estimator__min_impurity_split',
 'moc__estimator__min_samples_leaf',
 'moc__estimator__min_samples_split',
 'moc__estimator__min_weight_fraction_leaf',
 'moc__estimator__n_estimators',
 'moc__estimator__n_jobs',
 'moc__estimator__oob_score',
 'moc__estimator__random_state',
 'moc__estimator__verbose',
 'moc__estimator__warm_start',
 'moc__n_jobs',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'vect',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram

In [ ]:
parameters = {
    'vect__ngram_range':((1,1),(1,2)),
    'tfidf__use_idf': (True, False),
    'moc__estimator__n_estimators': [ 50, 100 ],
    'moc__estimator__max_depth': [ 3, 5 ],
}

cv = GridSearchCV(estimator = pipeline, param_grid = parameters, n_jobs= 12)

In [68]:
%time

cv.fit(X_train, Y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [11]:
cv.best_score_

0.4356759727699995

In [12]:
cv.best_params_

{'moc__estimator__max_depth': 3,
 'moc__estimator__n_estimators': 50,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [71]:
best_model = cv.best_estimator_

In [72]:
y_pred_train = best_model.predict(X_train)
print(classification_report(Y_train.values, y_pred_train, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               request       1.00      0.99      1.00      3572
                 offer       1.00      0.99      0.99        96
           aid_related       1.00      1.00      1.00      8673
          medical_help       1.00      0.99      1.00      1640
      medical_products       1.00      1.00      1.00      1057
     search_and_rescue       1.00      1.00      1.00       595
              security       1.00      0.98      0.99       371
              military       1.00      0.99      1.00       663
           child_alone       0.00      0.00      0.00         0
                 water       1.00      1.00      1.00      1366
                  food       1.00      1.00      1.00      2345
               shelter       1.00      1.00      1.00      1861
              clothing       1.00      1.00      1.00       324
                 money       1.00      1.00      1.00       487
        missing_people       1.00      

/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/

In [73]:
y_pred_test = best_model.predict(X_test)
print(classification_report(Y_test.values, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               request       0.82      0.49      0.61       902
                 offer       0.00      0.00      0.00        22
           aid_related       0.77      0.64      0.70      2187
          medical_help       0.63      0.06      0.11       444
      medical_products       0.75      0.11      0.18       256
     search_and_rescue       0.83      0.08      0.14       129
              security       0.00      0.00      0.00       100
              military       0.64      0.05      0.09       197
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.44      0.58       306
                  food       0.87      0.53      0.66       578
               shelter       0.81      0.36      0.50       453
              clothing       0.80      0.10      0.18        81
                 money       0.83      0.04      0.08       117
        missing_people       1.00      

/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(AdaBoostClassifier()))
])

In [8]:
cv = pipeline

### 9. Export your model as a pickle file

In [9]:
pickle.dump(cv, open('models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.